<a href="https://colab.research.google.com/github/Nixtla/hierarchicalforecast/blob/feat%2Fexample/examples/ReconcileFromBottomSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reconcile time series having only the lowest hierarchy

In many cases, only the time series at the lowest level of the hierarchies (bottom time series) are available. `HierarchicalForecast` has tools to create time series for all hierarchies. In this notebook we will see how to do it.

In [2]:
%%capture
!pip install hierarchicalforecast
!pip install statsforecast

In [21]:
import pandas as pd

#obtain hierarchical reconciliation methods and evaluation
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import BottomUp, MinTrace
from hierarchicalforecast.utils import hierarchize
# compute base forecast no coherent
from statsforecast.core import StatsForecast
from statsforecast.models import auto_arima, naive

## Aggregate bottom time series

In this example we will use the [Tourism](https://otexts.com/fpp3/tourism.html) dataset. The dataset only contains the time series at the lowest level, so we need to create the time series for all hierarchies.

In [7]:
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/tourism.csv')
df = df.rename({'Trips': 'y', 'Quarter': 'ds'}, axis=1)
df.insert(0, 'Country', 'Australia')
df = df[['Country', 'Region', 'State', 'Purpose', 'ds', 'y']]
df.head()

,Country,Region,State,Purpose,ds,y
0,Australia,Adelaide,South Australia,Business,1998 Q1,135.077690
1,Australia,Adelaide,South Australia,Business,1998 Q2,109.987316
2,Australia,Adelaide,South Australia,Business,1998 Q3,166.034687
3,Australia,Adelaide,South Australia,Business,1998 Q4,127.160464
4,Australia,Adelaide,South Australia,Business,1999 Q1,137.448533


In [23]:
pd.to_datetime(df['ds'], format='%Y ')

ParserError: ignored

The dataset can be grouped in the following non-strictly hierarchical structure.

In [9]:
hiers = [
    ['Country'],
    ['Country', 'State'], 
    ['Country', 'Purpose'], 
    ['Country', 'State', 'Region'], 
    ['Country', 'State', 'Purpose'], 
    ['Country', 'State', 'Region', 'Purpose']
]

Using the `hierarchize` function from `HierarchicalForecast` we can get the full set of time series.

In [12]:
Y_df, S, tags = hierarchize(df, hiers)

In [13]:
Y_df.head()

,ds,y
unique_id,,
Australia,1998 Q1,23182.197269
Australia,1998 Q2,20323.380067
Australia,1998 Q3,19826.640511
Australia,1998 Q4,20830.129891
Australia,1999 Q1,22087.353380


In [15]:
S.iloc[:5, :5]

,Australia/ACT/Canberra/Business,Australia/ACT/Canberra/Holiday,Australia/ACT/Canberra/Other,Australia/ACT/Canberra/Visiting,Australia/New South Wales/Blue Mountains/Business
Australia,1.0,1.0,1.0,1.0,1.0
Australia/ACT,1.0,1.0,1.0,1.0,0.0
Australia/New South Wales,0.0,0.0,0.0,0.0,1.0
Australia/Northern Territory,0.0,0.0,0.0,0.0,0.0
Australia/Queensland,0.0,0.0,0.0,0.0,0.0


In [17]:
tags['Country/Purpose']

array(['Australia/Business', 'Australia/Holiday', 'Australia/Other',
       'Australia/Visiting'], dtype=object)

## Computing base forecasts

The following cell computes the **base forecasts** for each time series in `Y_df` using the `auto_arima` and `naive` models. Observe that `Y_hat_df` contains the forecasts but they are not coherent.

In [22]:
%%capture
fcst = StatsForecast(df=Y_df, 
                     models=[naive], 
                     freq='M', n_jobs=-1)
Y_hat_df = fcst.forecast(h=12)

TypeError: ignored

## Reconcile forecasts

The following cell makes the previous forecasts coherent using the `HierarchicalReconciliation` class. Since the hierarchy structure is not strict, we can't use methods such as `TopDown` or `MiddleOut`. In this example we use `BottomUp` and `MinTrace`.

In [ ]:
reconcilers = [
    BottomUp(),
    MinTrace(method='ols'),
    MinTrace(method='wls_struct')
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df = hrec.reconcile(Y_hat_df, Y_df, S, tags)

The dataframe `Y_rec_df` contains the reconciled forecasts.

In [ ]:
Y_rec_df.head()